## POKEMON DETECTION

In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/pokemon-images-and-types'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pokemon-images-and-types/pokemon.csv
/kaggle/input/pokemon-images-and-types/images/throh.png
/kaggle/input/pokemon-images-and-types/images/samurott.png
/kaggle/input/pokemon-images-and-types/images/venomoth.png
/kaggle/input/pokemon-images-and-types/images/latios.png
/kaggle/input/pokemon-images-and-types/images/shinx.png
/kaggle/input/pokemon-images-and-types/images/durant.png
/kaggle/input/pokemon-images-and-types/images/primarina.png
/kaggle/input/pokemon-images-and-types/images/geodude.png
/kaggle/input/pokemon-images-and-types/images/popplio.png
/kaggle/input/pokemon-images-and-types/images/wingull.png
/kaggle/input/pokemon-images-and-types/images/miltank.png
/kaggle/input/pokemon-images-and-types/images/castform.png
/kaggle/input/pokemon-images-and-types/images/squirtle.png
/kaggle/input/pokemon-images-and-types/images/shaymin-land.png
/kaggle/input/pokemon-images-and-types/images/raikou.png
/kaggle/input/pokemon-images-and-types/images/guzzlord.png
/kaggle/input/po

In [16]:
data=pd.read_csv("/kaggle/input/pokemon-images-and-types/pokemon.csv")

In [17]:
data.head()

,Name,Type1,Type2,Evolution
0,bulbasaur,Grass,Poison,ivysaur
1,ivysaur,Grass,Poison,venusaur
2,venusaur,Grass,Poison,NaN
3,charmander,Fire,NaN,charmeleon
4,charmeleon,Fire,NaN,charizard


In [18]:
data=data.drop(columns=["Type2","Evolution"],axis=1)

In [19]:
data.head()

,Name,Type1
0,bulbasaur,Grass
1,ivysaur,Grass
2,venusaur,Grass
3,charmander,Fire
4,charmeleon,Fire


In [20]:
data["Name"]="/kaggle/input/pokemon-images-and-types/images/"+data["Name"]+".png"

In [21]:
data.head()

,Name,Type1
0,/kaggle/input/pokemon-images-and-types/images/...,Grass
1,/kaggle/input/pokemon-images-and-types/images/...,Grass
2,/kaggle/input/pokemon-images-and-types/images/...,Grass
3,/kaggle/input/pokemon-images-and-types/images/...,Fire
4,/kaggle/input/pokemon-images-and-types/images/...,Fire


In [22]:
x=data["Name"]
y=data["Type1"]

In [23]:
from tensorflow.keras.preprocessing import image

In [24]:
from sklearn.preprocessing import LabelEncoder

y_end=LabelEncoder().fit_transform(y)

In [25]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y_end,test_size=0.2,random_state=42)

In [26]:
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array
xtrain = np.array([load_and_preprocess_image(img) for img in xtrain])
xtest = np.array([load_and_preprocess_image(img) for img in xtest])

In [28]:
import tensorflow as tf
xtrain=np.array(xtrain)
ytrain=np.array(ytrain)
ytest=np.array(ytest)
xtest=np.array(xtest)
indices=np.arange(xtrain.shape[0])
np.random.shuffle(indices)
xtrain=xtrain[indices]
ytrain=ytrain[indices]

traindata=tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
traindata=traindata.shuffle(buffer_size=1024).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valdata=tf.data.Dataset.from_tensor_slices((xtest,ytest))
valdata=valdata.shuffle(buffer_size=1024).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report , accuracy_score , precision_score , recall_score

input_shape = (224, 224, 3)
# Create the input layer
input_layer = Input(shape=input_shape)

# Add layers to build the model
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(18, activation='softmax')(x)  # Correct the output layer

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy')
history = model.fit(traindata,epochs=25,validation_data=valdata,verbose=1)

Epoch 1/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 2.9804 - val_loss: 2.8162
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 2.7560 - val_loss: 2.7655
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 2.5086 - val_loss: 2.8416
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 2.0045 - val_loss: 2.8536
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 1.6096 - val_loss: 3.3685
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.9735 - val_loss: 3.6622
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.7683 - val_loss: 4.3572
Epoch 8/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.5186 - val_loss: 5.1114
Epoch 9/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.4562 - val_loss: 5.3881
Epoch 10/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.2970 - val_loss: 6.1683
Epoch 11/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - loss: 0.3077 - val_loss: 5.6856
Epoch 12/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 0.2145 - val_lo